In [1]:
import pandas as pd
import numpy as np

## **Step 1**: Grab and split the data
Here we fetch our training data (*retrieved from [here](https://www.kaggle.com/competitions/digit-recognizer/data)*) and break it down into:
- Features (`X`) - This is all of the inputs for our model. In this case, it's all 784 values of each pixel in the image from 0 (black) to 255 (white)
- Labels (`Y`) - This is the output/classification that we should get for each datapoint. In this case, it's the actual number that each image represents (*i.e. 0-9*)

Additionally we break off a piece of the overall training data for the **Validation Set**
> This validation set is not directly trained on like the rest of the training data, but is used to test how well the model responds to data it hasn't seen before. This helps prevent overfitting to the training data.

We are now have divided our training data into 4 pieces:
- Training Features (`X_train`)
- Training Labels (`Y_train`)
- Validation Features (`X_valid`)
- Validation Labels (`Y_valid`)

In [2]:
data = pd.read_csv('datasets/train.csv').values
M, n = data.shape

# 80-20 Split for training/validation sets
train_split = 0.2   
split_index = int(M*train_split)

data_train = data[split_index:].T
Y_train = data_train[0]
X_train = data_train[1:]

data_valid = data[:split_index].T
Y_valid = data_valid[0]
X_valid = data_valid[1:]